# 📊 Portfolio Analysis with Market Intelligence

## 🚀 **QUICK START GUIDE**

**This notebook analyzes your portfolio using the Market Intelligence System!** 

### 📁 **File Location:**
`/Users/vnktajay/Desktop/s&p500 market intelligence /market-intelligence-system/examples/example2/portfolio_analysis_notebook.ipynb`

### 🎯 **What This Notebook Does:**
1. **Loads Portfolio** - Reads your portfolio from `portfolio_snapshot.csv`
2. **Analyzes Stocks** - Runs comprehensive market intelligence on each stock
3. **Generates Reports** - Creates trading recommendations and analysis
4. **Displays Results** - Shows formatted portfolio summary and HTML dashboard

### ⚡ **How to Use:**
1. **Run Cell 1** - Install dependencies (first time only)
2. **Run Cell 2** - Setup environment and import modules
3. **Run Cell 3** - Load and display your portfolio
4. **Run Cell 4** - Run the market intelligence analysis
5. **View Results** - Portfolio summary and HTML dashboard appear below

### 📊 **Expected Results:**
- **Portfolio Analysis**: Complete analysis of all stocks in your portfolio
- **Trading Recommendations**: Buy/sell signals with entry/stop/target prices
- **HTML Dashboard**: Interactive charts and visualizations
- **Analysis Time**: 2-5 minutes depending on portfolio size

---

This notebook uses the Market Intelligence System to analyze your portfolio and provide actionable trading recommendations.


In [ ]:
# Portfolio Analysis - Dependencies Installation
import sys
import subprocess
from pathlib import Path

# Install dependencies for Market Intelligence
req_file = Path("../../requirements.txt")
extra_packages = [
    "beautifulsoup4>=4.12.0",
    "feedparser>=6.0.10", 
    "tweepy>=4.14.0",
    "sec-edgar-downloader>=5.0.0",
    "fredapi>=0.5.1",
    "lxml>=4.9.3",
    "html5lib>=1.1",
    "loguru>=0.7.0",
]

if req_file.exists():
    cmd = [sys.executable, "-m", "pip", "install", "-r", str(req_file)] + extra_packages
    print("Installing dependencies...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Dependencies installed successfully")
    else:
        print("⚠️ Some packages may already be installed")
else:
    print("⚠️ requirements.txt not found, installing extra packages only")
    cmd = [sys.executable, "-m", "pip", "install"] + extra_packages
    subprocess.run(cmd, check=False)

print(f"Python version: {sys.version}")


In [ ]:
# Environment Setup and Module Import
import os
import sys
import time
from pathlib import Path
import pandas as pd
from IPython.display import HTML, display, IFrame

# Add project root to Python path for imports
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import the market intelligence module
try:
    from src.comprehensive_market_intelligence import (
        ComprehensiveMarketIntelligence,
        IntelligenceConfig,
    )
    print("✅ Market Intelligence module imported successfully")
except ImportError as e:
    print(f"❌ Error importing module: {e}")
    print(f"Current directory: {Path.cwd()}")
    print(f"Project root: {project_root}")
    print("Make sure comprehensive_market_intelligence.py is in the src/ directory")

print(f"📁 Current directory: {Path.cwd()}")
print(f"📁 Project root: {project_root}")


In [ ]:
# Load and Display Portfolio
def load_portfolio(csv_path: Path) -> pd.DataFrame:
    """Load portfolio from CSV file."""
    if not csv_path.exists():
        raise FileNotFoundError(f"Portfolio file not found: {csv_path}")
    
    df = pd.read_csv(csv_path)
    if df.empty:
        raise ValueError(f"Portfolio file {csv_path} has no rows to analyse.")
    
    return df

def get_tickers_from_portfolio(df: pd.DataFrame) -> list:
    """Extract ticker symbols from portfolio DataFrame."""
    candidates = [
        col
        for col in df.columns
        if str(col).strip().lower() in {"ticker", "symbol", "tickers", "symbols"}
    ]
    
    if candidates:
        series = df[candidates[0]]
    else:
        series = df.iloc[:, 0]
    
    tickers = (
        series.dropna()
        .astype(str)
        .str.strip()
        .str.upper()
        .str.replace(".", "-", regex=False)
    )
    
    tickers = [ticker for ticker in tickers.tolist() if ticker]
    if not tickers:
        raise ValueError("No valid ticker symbols were found in the portfolio.")
    
    return tickers

# Load portfolio
portfolio_file = Path("portfolio_snapshot.csv")
print(f"📊 Loading portfolio from: {portfolio_file.absolute()}")

try:
    portfolio_df = load_portfolio(portfolio_file)
    tickers = get_tickers_from_portfolio(portfolio_df)
    
    print(f"✅ Portfolio loaded successfully!")
    print(f"📈 Found {len(tickers)} stocks in portfolio")
    print(f"🎯 Tickers: {', '.join(tickers)}")
    
    # Display portfolio summary
    print("\n📊 **PORTFOLIO SUMMARY:**")
    print("=" * 80)
    
    # Show key columns if they exist
    display_cols = ['Symbol', 'Qty', 'Market Value', 'Last', 'Open P&L %']
    available_cols = [col for col in display_cols if col in portfolio_df.columns]
    
    if available_cols:
        display(portfolio_df[available_cols])
    else:
        display(portfolio_df.head())
    
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error loading portfolio: {e}")
    print("Make sure portfolio_snapshot.csv exists in the current directory")


In [ ]:
# Run Portfolio Analysis
def run_portfolio_analysis(tickers: list, output_dir: Path) -> dict:
    """Run market intelligence analysis on portfolio tickers."""
    
    config = IntelligenceConfig(
        max_workers=min(4, max(1, len(tickers))),
        min_conviction_score=0.0,
        max_recommendations=len(tickers),
        min_price=0,
        min_volume=0,
        tickers_limit=len(tickers),
        custom_tickers=tickers,
        output_dir=output_dir,
    )

    print(f"🚀 Starting portfolio analysis...")
    print(f"📊 Analyzing {len(tickers)} stocks: {', '.join(tickers)}")
    print(f"⚡ Using {config.max_workers} parallel workers")
    print(f"⏱️  This will take 2-5 minutes...")
    
    start_time = time.time()
    
    engine = ComprehensiveMarketIntelligence(config)
    report = engine.generate_comprehensive_report()
    
    end_time = time.time()
    analysis_time = end_time - start_time
    
    print(f"\n✅ Analysis completed in {analysis_time:.1f} seconds ({analysis_time/60:.1f} minutes)")
    
    return report

# Run analysis if portfolio was loaded successfully
if 'tickers' in locals() and tickers:
    output_dir = Path("portfolio_analysis_output")
    
    try:
        report = run_portfolio_analysis(tickers, output_dir)
        
        # Display summary
        recommendations = report.get('recommendations', [])
        print(f"\n📊 **ANALYSIS RESULTS:**")
        print("=" * 60)
        print(f"📈 Stocks Analyzed: {len(tickers)}")
        print(f"🎯 Recommendations Generated: {len(recommendations)}")
        
        # Count recommendation types
        if recommendations:
            buy_count = len([r for r in recommendations if r.get('recommendation') == 'BUY'])
            strong_buy_count = len([r for r in recommendations if r.get('recommendation') == 'STRONG_BUY'])
            sell_count = len([r for r in recommendations if r.get('recommendation') == 'SELL'])
            
            print(f"⭐ Strong Buys: {strong_buy_count}")
            print(f"📈 Buy Recommendations: {buy_count}")
            print(f"📉 Sell Recommendations: {sell_count}")
            
            # Show top recommendations
            print("\n🏆 **TOP RECOMMENDATIONS:**")
            for i, rec in enumerate(recommendations[:5], 1):
                ticker = rec.get('ticker', 'N/A')
                company = rec.get('company', 'N/A')
                recommendation = rec.get('recommendation', 'N/A')
                conviction = rec.get('conviction_score', 0)
                entry_price = rec.get('entry_price', 'N/A')
                target_price = rec.get('target_price', 'N/A')
                
                print(f"  {i}. {ticker} - {recommendation} (Conviction: {conviction:.1f}%)")
                print(f"     {company} | Entry: ${entry_price:.2f} | Target: ${target_price:.2f}")
        
        print("\n📁 Generated Files:")
        print(f"📊 Dashboard: {output_dir}/dashboard_*.html")
        print(f"📄 JSON Report: {output_dir}/market_intelligence_*.json")
        print(f"📋 CSV Recommendations: {output_dir}/recommendations_*.csv")
        
        print("\n✅ Portfolio Analysis Complete!")
        
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ No portfolio loaded. Please run the previous cell first.")


In [ ]:
# Display HTML Dashboard Inline
if 'output_dir' in locals() and output_dir.exists():
    html_files = list(output_dir.glob("*.html"))
    
    if html_files:
        # Get the most recent HTML file
        latest_html = max(html_files, key=os.path.getctime)
        print(f"📈 Displaying HTML Dashboard: {latest_html.name}")
        
        # Read and display the HTML content inline
        with open(latest_html, 'r', encoding='utf-8') as f:
            html_content = f.read()
        
        # Display the HTML dashboard inline
        display(HTML(html_content))
        
        print(f"\n✅ HTML Dashboard displayed above!")
        print(f"📁 Full dashboard also saved to: {latest_html.absolute()}")
    else:
        print("❌ No HTML dashboard found. Make sure the analysis completed successfully.")
        print("📁 Check the portfolio_analysis_output/ folder for generated files.")
else:
    print("❌ No analysis output found. Please run the analysis first.")


In [ ]:
# Display Trading Recommendations Table
if 'report' in locals() and report:
    recommendations = report.get('recommendations', [])
    
    if recommendations:
        print(f"🎯 **PORTFOLIO TRADING RECOMMENDATIONS** ({len(recommendations)} total)")
        print("=" * 100)
        
        # Create a DataFrame for better display
        rec_df = pd.DataFrame(recommendations)
        
        # Select key columns for display
        display_cols = ['ticker', 'recommendation', 'conviction_score', 'entry_price', 
                       'stop_loss_price', 'target_price', 'risk_reward_ratio', 'sector']
        
        # Filter to available columns
        available_cols = [col for col in display_cols if col in rec_df.columns]
        
        if available_cols:
            # Format the display
            display_df = rec_df[available_cols].copy()
            
            # Format numeric columns
            if 'entry_price' in display_df.columns:
                display_df['entry_price'] = display_df['entry_price'].apply(lambda x: f"${x:.2f}" if pd.notna(x) else "N/A")
            if 'stop_loss_price' in display_df.columns:
                display_df['stop_loss_price'] = display_df['stop_loss_price'].apply(lambda x: f"${x:.2f}" if pd.notna(x) else "N/A")
            if 'target_price' in display_df.columns:
                display_df['target_price'] = display_df['target_price'].apply(lambda x: f"${x:.2f}" if pd.notna(x) else "N/A")
            if 'conviction_score' in display_df.columns:
                display_df['conviction_score'] = display_df['conviction_score'].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "N/A")
            if 'risk_reward_ratio' in display_df.columns:
                display_df['risk_reward_ratio'] = display_df['risk_reward_ratio'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else "N/A")
            
            # Display the table
            display(display_df)
            
            print(f"\n📊 **SUMMARY:**")
            print(f"   • Total Recommendations: {len(recommendations)}")
            print(f"   • Buy Signals: {len([r for r in recommendations if r.get('recommendation') == 'BUY'])}")
            print(f"   • Sell Signals: {len([r for r in recommendations if r.get('recommendation') == 'SELL'])}")
            print(f"   • Strong Buys: {len([r for r in recommendations if r.get('recommendation') == 'STRONG_BUY'])}")
            
        else:
            print("❌ No recommendation data available for display")
    else:
        print("❌ No recommendations found in the report")
else:
    print("❌ No report data available. Make sure to run the analysis first.")


## 📊 Portfolio Analysis Results

The portfolio analysis has completed successfully! The results are displayed above:

### 🎯 **What You'll See Above:**
1. **📈 HTML Dashboard** - Interactive charts and visualizations (displayed inline)
2. **📋 Trading Recommendations Table** - Detailed buy/sell signals with prices
3. **📁 Saved Files** - Complete reports in the `portfolio_analysis_output/` folder

### 🎯 **Trading Details Included:**
- **Entry Price**: Exact entry point for each stock
- **Stop Loss**: Risk management level (2×ATR below entry)  
- **Target Price**: Profit target (3×ATR above entry)
- **Exit Plan**: Complete strategy with holding period
- **Risk/Reward Ratio**: Calculated risk-reward ratio
- **Technical Drivers**: RSI, moving averages, patterns

### 📁 **Generated Files:**
- **HTML Dashboard**: Interactive dashboard with charts and trading details
- **JSON Report**: Complete structured data
- **CSV Recommendations**: Detailed buy/sell recommendations with entry prices, stop losses, and target prices

### 💡 **How to Use the Results:**
1. **Review Recommendations**: Check the conviction scores and trading signals
2. **Analyze Entry Points**: Use the entry prices for timing your trades
3. **Set Stop Losses**: Use the stop loss prices for risk management
4. **Plan Exits**: Use the target prices and exit strategies
5. **Monitor Performance**: Track your portfolio against the recommendations

### ⚠️ **Important Notes:**
- This analysis is for informational purposes only
- Always do your own research before making investment decisions
- Past performance does not guarantee future results
- Consider your risk tolerance and investment goals
